In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path

[
    sys.path.append(str(path))
    for path in [Path.cwd(), Path.cwd().parent, Path.cwd().parent / "python"]
    if str(path) not in sys.path
]

[None, None]

In [4]:
import os

os.environ["OPENAI_API_KEY"] = "foo"
from langchain.base_language import BaseLanguageModel
from langchain.llms import OpenAI
from pydantic.v1 import BaseModel


class Foo(BaseModel):
    llm: BaseLanguageModel | None = None


llm = OpenAI()
# Works
Foo()
# Fails
Foo(llm=llm)

Foo(llm=OpenAI(client=<openai.resources.completions.Completions object at 0x7f8addf1f9d0>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7f8ade7ef2b0>, openai_api_key='foo', openai_proxy=''))

In [9]:
class Car:
    def __init__(self, brand, model):
        self.brand = brand
        self.model = model


# Create a Car object
my_car = Car("Toyota", "Camry")

# Dynamically add a new attribute

my_car.year = 2022


print(my_car.year)  # Output: 2022

2022


In [13]:
hasattr(my_car, "year")

True

In [5]:
from typing import Type, TypeVar

from pydantic import BaseModel, create_model

T = TypeVar("T", bound=BaseModel)


def add_field_to_class(
    base_class: Type[T], field_name: str, field_type: Type, required: bool = False
) -> Type[T]:
    """
    Creates a new Pydantic class by inheriting from the provided base class
    and adding a new field with the specified name and type.

    Args:
        base_class (Type[T]): The base Pydantic class to inherit from.
        field_name (str): The name of the new field to add.
        field_type (Type): The type of the new field.
        required (bool, optional): Whether the new field should be required or not. Defaults to False.

    Returns:
        Type[T]: A new Pydantic class with the additional field.
    """
    field_definition = (field_type, ...) if required else (field_type, None)
    new_class_name = f"{base_class.__name__}With{field_name.capitalize()}"

    return create_model(
        new_class_name,
        **{field_name: field_definition},
        __base__=base_class,
    )


class Car(BaseModel):
    brand: str
    model: str
    year: int


car1 = Car(brand="Toyota", model="Camry", year=2022)
CarWithColor = add_field_to_class(Car, "color", str, required=True)
my_car = CarWithColor(brand="Toyota", model="Camry", year=2022, color="Red")

In [23]:
from typing import Any


def add_field_to_obj(obj: BaseModel, field_name: str,  value: Any) -> BaseModel:
    new_cls = add_field_to_class(type(obj), field_name, type(value), True)
    new_obj = new_cls.model_validate(my_car.model_dump() | {field_name: value})
    return new_obj


new_car = add_field_to_obj(my_car, "llm_id", "LLMID")
new_car

CarWithColorWithLlm_id(brand='Toyota', model='Camry', year=2022, color='Red', llm_id='LLMID')

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given